In [2]:
import glob, re, pandas as pd, fitz, xlsxwriter, string, os, sys
from pathlib import Path

# Struktur kata kunci dan grupnya
KEYWORDS_GROUPS = {
    "304-1": "Biodiversity",
    "304-2": "Biodiversity",
    "304-3": "Biodiversity",
    "304-4": "Biodiversity",
}

# Fungsi untuk membersihkan teks dari karakter ilegal
def remove_illegal_characters(text):
    printable = set(string.printable)
    text = "".join(filter(lambda x: x in printable, text))
    illegal_chars = r"[\000-\010]|[\013-\014]|[\016-\037]"
    return re.sub(illegal_chars, "", text)

# Fungsi untuk mendapatkan semua file PDF dalam suatu folder
def get_all_pdf_files(folder_path):
    return [os.path.join(root, file) for root, _, files in os.walk(folder_path) for file in files if file.lower().endswith(".pdf")]

# Fungsi untuk mencari kata kunci dalam file PDF
def search_keywords_in_pdf(file_path, df_results,tahun):
    file_name = os.path.basename(file_path)
    ticker = file_name.split("-")[1]
    # tahun = file_name.split("_")[1]
    found = False
    try:
        doc = fitz.open(file_path)
        for page_num, page in enumerate(doc):
            text = page.get_text()
            text = remove_illegal_characters(text)
            for keyword in KEYWORDS_GROUPS.keys():
                if re.search(r"\b" + re.escape(keyword) + r"\b[\s.,;:!?)]*", text, re.IGNORECASE):
                    found = True
                    new_row = pd.DataFrame({
                        "Ticker": [ticker],
                        "Tahun": [tahun],
                        "PDF files": [file_path],
                        "Keyword": [keyword],
                        "Page": [page_num + 1],
                        "Text": [text]
                    })
                    df_results = pd.concat([df_results, new_row], ignore_index=True)
        if not found:  # Jika tidak ada kata kunci yang ditemukan dalam seluruh dokumen
            new_row = pd.DataFrame({
                "Ticker": [ticker],
                "Tahun": [tahun],
                "PDF files": [file_path],
                "Keyword": [None],
                "Page": [None],
                "Text": [None]
            })
            df_results = pd.concat([df_results, new_row], ignore_index=True)
        doc.close()
    except Exception as e:
        print(f"Error processing PDF: {file_path}, Error: {str(e)}")
    return df_results

folder_tahun = ['2022']

main_folder_path = Path("D:/CESGS")
output_folder_path = Path("D:/CESGS")

for t in folder_tahun:
    folder_path = main_folder_path / t
    output_fp = output_folder_path / f"[O] {t}"
    # Check if the directory exists before making a new directory
    if not output_fp.exists():
        output_fp.mkdir()
    # for i in folder_isi:
    df_results = pd.DataFrame(columns=["Ticker", "Tahun", "PDF files", "Keyword", "Page", "Text"])
    folder_pdf = folder_path
    pdf_files = get_all_pdf_files(folder_pdf)
    for file_path in pdf_files:
        df_results = search_keywords_in_pdf(file_path, df_results, t)
        print(f"Processed file: {file_path}")
    excel_path = output_fp / f"Biodiversity.xlsx"
    df_results.to_excel(excel_path, index=False)
    print("Results saved to Excel.")


Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-AALI-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ABMM-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ACES-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ADCP-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ADES-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ADHI-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ADMG-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ADMR-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-ADRO-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-AGAR-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-AHAP-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_Rename\ID-AISA-SR-2022.pdf
Processed file: D:\CESGS\2022\Sustainability Report_